In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input,Activation
from keras.layers import LSTM,SimpleRNN
from keras import optimizers

Using TensorFlow backend.


In [3]:
data = pd.read_csv("Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",usecols=None)

In [4]:
df =  pd.DataFrame(data)
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,22,1266342,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,22,1319353,41,44,2664,6954,456,0,64.975610,109.864573,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,22,160,1,1,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,22,1303488,41,42,2728,6634,456,0,66.536585,110.129945,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,35396,77,1,2,0,0,0,0,0.000000,0.000000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [5]:
def return_classes(df,need=False):
    le = LabelEncoder()
    le.fit(df)
    classes = le.classes_.tolist()
    if need:
        d={}
        for i,clas in enumerate(le.classes_.tolist()):
            c = le.transform([clas])
            d[i]=list(le.inverse_transform(c))[0]
        return le.transform(df),d
    return le.transform(df)

In [6]:
df.iloc[:,-1],mapping = return_classes(df.iloc[:,-1],True)
print(mapping)

{0: 'BENIGN', 1: 'PortScan'}


In [7]:
x = len(df.iloc[:,-1].unique())
print(x)

2


In [8]:
X_data = df.iloc[:,:-2].values
y_data = df.iloc[:,-1].values

In [9]:
X_train,X_val,y_train,y_val = train_test_split(X_data,y_data,test_size=0.2)

In [10]:
y_train =np_utils. to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)

In [11]:
#scaler = Normalizer().fit(X_train)
#X_train = scaler.transform(X_train)

#scaler = Normalizer().fit(X_val)
#X_val = scaler.transform(X_val)

In [12]:
X_train = np.reshape( X_train, ( X_train.shape[0], 1,  X_train.shape[1]))
X_val = np.reshape( X_val , ( X_val .shape[0], 1,  X_val .shape[1]))
print(X_train.shape,X_val.shape)
print(y_train.shape,y_val.shape)

(229173, 1, 77) (57294, 1, 77)
(229173, 2) (57294, 2)


In [13]:
from keras.layers import Flatten
model = Sequential()
model.add(LSTM(80, input_shape=(1,40),return_sequences=True))
#model.add(TimeDistributed(Dense(1)))
#model.add(AveragePooling1D())

model.add(Flatten())
model.add(Dense(23, activation='softmax'))

In [14]:
from keras.layers import Flatten
model2 = Sequential()
model2.add(LSTM(256, input_shape=(1,77),return_sequences=True))
#model.add(TimeDistributed(Dense(1)))
#model.add(AveragePooling1D())

model2.add(Flatten())
model2.add(Dense(x, activation='softmax'))

In [15]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 1, 256)            342016    
_________________________________________________________________
flatten_2 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 342,530
Trainable params: 342,530
Non-trainable params: 0
_________________________________________________________________


In [19]:
#model2.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [21]:
from keras import metrics
sgd = optimizers.SGD(lr=0.01)
model2.compile(loss='mean_squared_error',optimizer='sgd', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [22]:
model2.fit(X_train,y_train,epochs=1,validation_data=(X_val,y_val))

Train on 229173 samples, validate on 57294 samples
Epoch 1/1
229173/229173 [==============================] - 63s - loss: nan - acc: 0.4452 - val_loss: nan - val_acc: 0.4454
